In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

URL = "https://roll20.net/compendium/dnd5e/Index%3ASpells/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
all_spells = {}
results = soup.findAll("a")
for spell in results:
    if spell['href'].startswith('/compendium'):
        # print(str(item.string) + ": " + str(item['href']))
        
        spell = {
            "Spell": str(spell.string).lstrip(),
            "Spell Roll20 Link": "https://roll20.net" + str(spell['href'])
        }
        all_spells[spell["Spell"]] = spell
        
failed_keys = []
notice_every = 10
notice_count = 0
notice_tmp = 0
for key in all_spells:
    item = all_spells[key]
    if str(item["Spell Roll20 Link"]) != "https://roll20.net/compendium/dnd5e/Items:Emblem":
        try:
            item_page = requests.get(item["Spell Roll20 Link"])
            item_soup = BeautifulSoup(item_page.content, "html.parser")
            item_content = item_soup.find('div', {"id": "pagecontent"})
            desc = ""
            for s in item_content.contents:
                if not str(s).startswith("<"):
                    if not str(s).startswith("."):
                        desc = desc + " "
                    desc = desc + str(s).strip()
                elif str(s.string) != "None":
                    desc = desc + " " + str(s.string)
            desc = desc.strip()
            if desc == '':
                desc = "No Description"
            item["Description"] = desc

            item_attrs = item_soup.findAll('div', {"class": "row attrListItem"})
            for attr in item_attrs:
                name = str(attr.find("div", {"class": "col-md-3 attrName"}).string)
                val = str(attr.find("div", {"class": "col-md-7 attrValue"}).find("div", {"class": "value"}).string)
                #print("Name: " + name + "  val: " + val)
                if val.isnumeric():
                    val = int(val)
                elif isfloat(val):
                    val = float(val)
                item[name] = val
                
            if notice_tmp >= notice_every:
                notice_tmp = 0
                notice_count += 1
                print("Still Running. Notice: " + str(notice_count))
            else:
                notice_tmp += 1
        except KeyError:
            print("KEY ERROR:")
            print(item)
            failed_keys.append(key)
        except AttributeError:
            print("ATTRIBUTE ERROR:")
            print(item)
            failed_keys.append(key)
            
failed_spells = []
for k in failed_keys:
    failed_spells.append(all_spells.pop(k))
    
with open("scraper_output/roll_20_all_spells.json", "w", encoding='utf-8') as outfile:
    json.dump(all_spells, outfile, ensure_ascii=False, indent=4)
    
print("")
print("")
print("Failed Keys:")
print(failed_keys)
print("")
print("Failed Spells:")
print(failed_spells)

Still Running. Notice: 1
ATTRIBUTE ERROR:
{'Spell': 'Antipathy Sympathy', 'Spell Roll20 Link': 'https://roll20.net/compendium/dnd5e/Spells:Antipathy%20Sympathy'}
Still Running. Notice: 2
ATTRIBUTE ERROR:
{'Spell': 'Blindness Deafness', 'Spell Roll20 Link': 'https://roll20.net/compendium/dnd5e/Spells:Blindness%20Deafness'}
Still Running. Notice: 3
Still Running. Notice: 4
Still Running. Notice: 5
Still Running. Notice: 6
Still Running. Notice: 7
Still Running. Notice: 8
ATTRIBUTE ERROR:
{'Spell': 'Enlarge Reduce', 'Spell Roll20 Link': 'https://roll20.net/compendium/dnd5e/Spells:Enlarge%20Reduce'}
Still Running. Notice: 9
Still Running. Notice: 10
Still Running. Notice: 11
Still Running. Notice: 12
Still Running. Notice: 13
Still Running. Notice: 14
Still Running. Notice: 15
Still Running. Notice: 16
Still Running. Notice: 17
Still Running. Notice: 18
Still Running. Notice: 19
Still Running. Notice: 20
Still Running. Notice: 21
Still Running. Notice: 22
Still Running. Notice: 23
Still Ru